# Introduction
Ce notebook utilise des données fournies par le [Musée Guimet](https://www.guimet.fr/fr) dans le cadre du [projet HikarIA](https://www.guimet.fr/fr/actualites-du-musee/le-projet-hikaria-laureat-de-france-2030), un projet de recherche visant à développer un système de cataloguage automatique d'une collection d'albums de photographies japonaises du XIXème. 

Ce premier TP utilise un ensemble de tags (`tags.txt`) utilisés pour décrire les photographies.

<img src="https://storage.teklia.com/shared/deepnlp-labs/images/16-519781.jpg" width="200"/>








## Embeddings statiques Word2Vec



Dans cette section, nous allons explorer un modèle de plongement de mots pré-entraîné de type Word2Vec.
Nous allons utiliser un modèle Word2Vec pré-entraîné sur le corpus français Wac. 
Ce modèle a été entraîné sur un corpus de 1 milliard de mots en français. 

Cet embedding est disponble sous 2 formats:
- un format texte qui permet d'explorer facilement le modèle :
    - [frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.txt](https://storage.teklia.com/shared/deepnlp-labs/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.txt)
- un format binaire qui peut être chargé avec la librairie Gensim : 
    - [frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin](https://storage.teklia.com/shared/deepnlp-labs/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin)

### Embedding de mot

Téléchargez le fichier texte sur votre machine pour l'analyser.

#### Question : 
>* Donner la taille des fichiers d'embeddings
>* En explorant le contenu du fichier d'embedding au format texte, donner le nombre de mots pour lesquels ce modèle  fournit des embeddings et la taille de l'embedding pour chacun des mots




### Similarité de mots

Nous allons maintenant utiliser la librairie [Gensim](https://radimrehurek.com/gensim/) pour charger le modèle Word2Vec et l'utilser. 

#### Question : 
>* Modifier le code suivant pour charger le fichier modèle Word2Vec au format binaire
>* Choisissez quelques exemples dans le fichier tags.csv et utilisez la fonction [most_similar](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) pour trouver les mots les plus proches selon le modèle
>* Afin de deviner le sens des mots "yokohama", "kanto" et "shamisen", cherchez leurs plus proches voisins. Expliquez les résultats.



In [ ]:
# Download the French word embeddings
!wget "https://storage.teklia.com/shared/deepnlp-labs/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin"-O frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin


In [ ]:
from gensim.models import KeyedVectors

## YOUR CODE HERE
embedding_file =""
model = KeyedVectors.load_word2vec_format(embedding_file, binary=True, unicode_errors="ignore")
## YOUR CODE HERE


### Arithmétique sémantique

Une des propriétés les plus originales des embeddings Word2Vec est que les relations sémantiques entre les vecteurs peuvent être modélisées par des opérations arithmétiques. Etant données les vecteurs repésentant les mots `roi`, `homme` et `femme`, il est possible de calculer le vecteur `v` comme :  

`v = vecteur(roi)-vecteur(homme)+vecteur(femme)`

Cette opération correspond à la relation sémantique suivante : *Le roi est à l'homme ce que la reine est à la femme* ce qui se traduit par l'arithmétique suivante : *le concept de roi, moins le concept d'homme  plus le concept de femme donne le concept de reine*.

En effet, en cherchant dans l'embedding, le mot dont le vecteur le plus proche est `v`, on trouve `reine`

#### Question : 
>* en utilisant la fonction [most_similar](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) en spécifiant les arguments `positive` pour les vecteurs à additionner et `negative` pour les vecteurs à soustraire, vérifier la relation *le concept de roi, moins le concept d'homme  plus le concept de femme donne le concept de reine*
>* Avec la même méthode, trouvez XXX dans les relations sémantiques suivantes
>   * *Paris est à la France ce que XXX est au Japon*
>   * *Chevalier est à la France ce qui XXX est au Japon*



In [ ]:
## YOUR CODE HERE


### Embedding d'une séquence de mots
Les embeddings Word2Vec permettent de représenter des mots sous forme vectorielle. Pour représenter des séquences de mots (expressions, phrases, documents), il faut  donc combiner les vecteurs de chacun des mots pour obtenir une représentation vectorielle de la séquence. La technique la plus simple consiste à prendre la moyenne des vecteurs des mots de la séquence. Cette approche a cependant l'inconvient de ne pas prendre en compte l'ordre des mots et d'être sensible aux points abérants (outliers).

#### Question : 
>* en utilisant la fonction [get_mean_vector](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.get_mean_vector) calculer les représentations vectorielles des phases suivantes : 
>   * *le cerisier est en fleurs*
>   * *le jardin est fleuri*
>   * *la femme joue de la musique*
>* en utilisant la fonction [cosine_similarities](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.cosine_similarities), calculer les similarités entre la première phrase et les deux autres. En déduire quelles phrases sont les plus proches selon le modèle.


In [ ]:
 # YOUR CODE HERE
phrase1 = "le cerisier est en fleurs"
phrase2 = "le jardin est fleuri"
phrase3 = "la femme joue de la musique"




### Représentation graphique

Les embeddings permettent de représenter les mots dans un espace en grande dimension (200, 756, 1500, etc). L'utilisation d'un espace en grande dimension est utile pour réprésenter les différents sens des mots et leurs caractéristiques grammaticales, mais elle rend la visualisation complexe. Afin de visualiser les positions relatives des mots, il faut réduire la dimension des embedding pour les afficher en dimension 2 par exemple. Il faut que la réduction de dimension respecte la répartition initiale des points dans l'espace pour que la visualisation en 2D garde un sens. Différentes techniques de [réduction de dimension](https://cedric.cnam.fr/vertigo/Cours/ml/coursReductionDimension.html) sont utilisables (ACP, LLE, t-SNE). Nous allons utiliser t-SNE.

#### Question : 
>* Explorer la visalisation t-SNE, noter quelques clusters de mots sémantiquement similaires.

In [ ]:
# Download the tags file
!wget https://storage.teklia.com/shared/deepnlp-labs/tags.txt -O tags.txt


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import plotly.express as px

# Load the tags
tags_file = 'tags.txt'
tags = [tag.strip() for tag in open(tags_file, 'r')]

# Take a subset of the tags of 200 words
tag_subset = tags[:200]

# Compute the vectors for the tags
vectors = np.array([model[tag] for tag in tag_subset])

# reduce the dimensionality of the vectors with t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_results = tsne.fit_transform(vectors)

# Prepare the data for visualisation
df = pd.DataFrame(tsne_results, columns=['Dimension 1', 'Dimension 2'])
df['Tag'] = tag_subset
# Visualise the tags
fig = px.scatter(df, x='Dimension 1', y='Dimension 2', text='Tag', title='Visualisation des tags avec t-SNE')
fig.update_traces(textposition='top center')

fig.show()


## Embedding dynamique BERT
### Le modèle Camembert
Dans cette section, nous allons étudier le modèle BERT et particulièrement un modèle BERT entrainé sur un corpus français, le modèle [Camembert](https://camembert-model.fr/). 

Pour utiliser le modèle, nous allons utiliser la librairie `transformers` de [HuggingFace](https://huggingface.co/docs/transformers/index). Cette librairie permet d'utiliser très facilement un très grand nombre de modèles avec les mêmes interfaces. 

L'utilisation du modèle nécessite le chargement de deux composants : 
- le modèle lui-même `CamembertForMaskedLM.from_pretrained('camembert-base')`
- le tokenizer, qui permet de découper les textes en tokens utilisables par le modèle pour faire des prédictions: `AutoTokenizer.from_pretrained('camembert-base')`

Chargeons ces deux composants et affichons la descriptions du modèle : 
- nombre de répétitions du bloc d'encodeur
- la définition du premier bloc, qui contient l'embedding initial des tokens

#### Question : 
>* en lisant la description du premier bloc d'encodeur, donner le nombre de tokens modélisés par le ce modèle et la taille des embeddings.
>* vérifier que vous retouvez les informations données dans la [documentation sur HuggingFace](https://huggingface.co/docs/transformers/en/model_doc/camembert#transformers.CamembertConfig)


In [ ]:
#!pip install transformers
#!pip install torch
from transformers import AutoModelForSequenceClassification, CamembertForMaskedLM, AutoTokenizer, AutoConfig
import torch

# Import the model and tokenizer
camembert = CamembertForMaskedLM.from_pretrained('camembert-base')
tokenizer = AutoTokenizer.from_pretrained('camembert-base')
# Print a description of the model
print(f"Ce modèle Camembert est composé de {len(camembert.roberta.encoder.layer)} blocs d'encodeur\n")
print ("Description de l'embedding initial :")
print (camembert.roberta.embeddings)
print("Composition de la première couche :")
print (camembert.roberta.encoder.layer[0])


### Prédiction en masquage

Le modèle est entrainé, sur une grande quantité de texte en français, à prédire des mots masqués dans la phrase en fonction du contexte. Le modèle doit donc apprendre le sens et la séquence des mots pour prédire les mots les plus probables étant donné un contexte. Une fois entrainé, le modèle peut donc prédire des mots masqués dans une phrase.

La première étape consiste à découper la phrase en utilisant les tokens modélisés par le modèle. La taille maximale d'une phrase que le modèle peut traiter est fixe. Les phrases sont donc complétées avec des token de `padding` pour atteindre cette taille.

#### Question : 
>* Executer la cellule suivante et donner :
>   * le nombre maximal de tokens dans une phrase traitée par le modèle
>   * le nombre de tokens utilisés pour encoder la phrase `Les cerisiers sont en`



In [ ]:
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns

# Define the sentence to complete
batch_sentences = [
    "Les cerisiers sont en <mask>",
    "Le samourail sort son <mask>",
    "Tokyo est la capitale du <mask>",
]
# Tokenize the sentences
tokenizer_output = tokenizer(
    batch_sentences,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

# Print the tokenized sentences
print ("nombres de tokens :")
print (len(tokenizer_output["input_ids"][0]))
print ("identifiants des tokens de la première phrase :")
print (tokenizer_output["input_ids"][0])
print ("tokens de la première phrase :")
tokens = tokenizer.convert_ids_to_tokens(tokenizer_output["input_ids"][0])
print(tokens)



Nous allons ensuite utiliser le modèle pour prédire les mots masqués. La prédiction est réalisée en appelant le modèle (fonction `__call__` en python). Cet appel prend la phrase tokenisée en entrée et retourne une structure qui contient : 
- la loss (None, car nous utilsons `torch.no_grad()`, pas besoin de calculer la loss pour une inférence)
- les logits, c'est à dire les scores de chaque token possible à chaque position
- les couches cachées du modèle (si on demande `output_hidden_states=True`)

#### Question : 
>* Executer la cellule suivante et expliquer à quoi correspondent les 3 dimensions de la variable `logit`.




In [ ]:

with torch.no_grad():
    model_output = camembert(**tokenizer_output)
pprint(model_output)
pprint(model_output.logits.shape)

La variable `logits`contient les scores de chaque token possible à chaque position dans la phrase. Ces scores ne sont pas normalisés, pour les transformer en probabilités (entre 0 et 1), nous utilisons une [fonction softmax](https://en.wikipedia.org/wiki/Softmax_function). Puis la fonction  `topk` de la [librairie torch](https://pytorch.org/docs/stable/generated/torch.topk.html) nous donne les index des 5 plus grandes probabilités. Il reste ensuite à transformer les index en token avec la fonction `convert_ids_to_tokens`.

#### Question : 
>* Modifier le code suivant pour donner les 5 mots les plus probables pour chacune des phrases à compléter.


In [ ]:
# print the logits for the masked token in the first sentence
masked_index = 6
sentence_logits = model_output.logits[0, masked_index]
# get the probabilities by applying the softmax function
probs = torch.nn.functional.softmax(sentence_logits, dim=0)
# get the top 3 predictions
values, predictions = torch.topk(probs, 3)
# print the top 3 predictions
for value, prediction in zip(values, predictions):
    token = tokenizer.convert_ids_to_tokens([prediction])[0]
    print(f"{token}: {value:.4f}")